In [1]:
"""
CREATE MOCK PARALLAX, METALLICITY, MASS AND PHOTOMETRY DATA USING ISOCHRONES (NO DUST)
"""
import numpy as np
import dill as dill
import AstroMethods as am
import pandas as pd

### USER-DEFINED PARAMETERS #########################################################
# datafile needs column headings with at least ra and dec, and additionally j, h, k, 
# teff, logg, m_h, mass depending on what's available
NMOCK        = 100
AGEMIN       = 0.5    # Gyr
AGEMAX       = 13.5
FEHMIN       = -1.0  # dex
FEHMAX       = 0.2
LMIN         = 0.    # rad
LMAX         = 2.*np.pi 
BMIN         = -np.pi/2. # rad
BMAX         = np.pi/2.
DISTMIN      = 0.1   # kpc
DISTMAX      = 20.
FCOORDERR    = 0.001
FVARPIERR    = 0.01
FFEHERR      = 0.01
FMASSERR     = 0.01
FTEFFERR     = 0.001
FLOGGERR     = 0.001
FMAGERR      = 0.0001
MOCKDATAFILE = "../results/mock_isochrone_data_nmock"+str(NMOCK)+".csv" 

## Draw random age, metallicity, and distance data (do mass later according to age and metallicity)

In [2]:
np.random.seed(5)
age  = np.random.uniform(AGEMIN,AGEMAX,NMOCK)
feh  = np.random.uniform(FEHMIN,FEHMAX,NMOCK)
l    = np.random.uniform(LMIN,LMAX,NMOCK)
b    = np.random.uniform(BMIN,BMAX,NMOCK)
dist = np.random.uniform(DISTMIN,DISTMAX,NMOCK)

## Estimate observed data using isochrone interpolant

In [3]:
# Undill Parsec isochrones and interpolants
print(" ")
print("Undilling isochrones and interpolants...")
with open("/home/payel/Dropbox/ResearchProjects/shared/python/ParsecIsochronesAgeDistance/data/stellarprop_parsecdefault_currentmass_m0mugrizJHK.dill", "rb") as input:
        pi = dill.load(input)
print("...done.")
print(" ")

 
Undilling isochrones and interpolants...
...done.
 


In [4]:
mass   = np.zeros(NMOCK)
output = np.zeros([NMOCK,24]) 
isoagegrid = np.copy(pi.isoage)
isomhgrid  = np.copy(pi.isomh)
for jmock in range(NMOCK):

    # Find closest isochrone interpolant for each star
    dage       = np.abs(isoagegrid - age[jmock])
    jage       = np.where(dage == np.min(dage))[0][0]
    dfeh       = np.abs(isomhgrid - feh[jmock])
    jfeh       = np.where(dfeh == np.min(dfeh))[0][0]

    # Use isochrones to estimate observed data
    interpname = "age"+str(np.round(isoagegrid[jage],8))+"mh"+str(isomhgrid[jfeh])
    if (interpname in pi.isointerpdict):
        isochrone = pi.isointerpdict[interpname]  
    else:
        interpname = "age"+str(np.round(isoagegrid[jage],9))+"mh"+str(isomhgrid[jfeh])
        if (interpname in pi.isointerpdict):
            isochrone = pi.isointerpdict[interpname] 
        else:
            interpname = "age"+str(np.round(isoagegrid[jage],10))+"mh"+str(isomhgrid[jfeh])
            if (interpname in pi.isointerpdict):
                isochrone = pi.isointerpdict[interpname]  
            else:
                interpname = "age"+str(np.round(isoagegrid[jage],11))+"mh"+str(isomhgrid[jfeh])
                if (interpname in pi.isointerpdict):
                    isochrone = pi.isointerpdict[interpname]  
                else:
                    interpname = "age"+str(np.round(isoagegrid[jage],12))+"mh"+str(isomhgrid[jfeh])
                    if (interpname in pi.isointerpdict):
                        isochrone = pi.isointerpdict[interpname]  
                    else:
                        interpname   = "age"+str(np.round(isoagegrid[jage],13))+"mh"+str(isomhgrid[jfeh])
                        if (interpname in pi.isointerpdict):
                            isochrone = pi.isointerpdict[interpname]  
                        else:
                            interpname = "age"+str(np.round(isoagegrid[jage],14))+"mh"+str(isomhgrid[jfeh])                 
    isochrone   = pi.isointerpdict[interpname]
    massmin     = pi.massmindict[interpname]*1.01
    massmax     = pi.massmaxdict[interpname]*0.99
    mass[jmock] = np.random.uniform(massmin,massmax)
                
    # Isochrone estimate of observables
    stellarprop = isochrone(mass[jmock]) # outputs initial mass, actual mass, log(Lum/Lsun), logTe,
    # log g, u, g, r, i, z, J, H, Ks
    modmass = stellarprop[1]    
    modteff = 10.**(stellarprop[3])
    modlogg = stellarprop[4]
    modabsJ = stellarprop[10]
    modabsH = stellarprop[11]
    modabsK = stellarprop[12]
    
    # Parallax (mas because distance in kpc)
    modvarpi = 1./dist[jmock]
    
    # Add noise
    eobsvarpi = FVARPIERR*modvarpi
    obsvarpi = np.random.normal(modvarpi,eobsvarpi)
    s        = 1./modvarpi
    eobsl    = np.abs(FCOORDERR*l[jmock])
    obsl     = np.random.normal(l[jmock],eobsl)
    eobsb    = np.abs(FCOORDERR*b[jmock])
    obsb     = np.random.normal(b[jmock],eobsb)
    eobsfeh  = np.abs(FFEHERR*isomhgrid[jfeh])
    obsfeh   = np.random.normal(isomhgrid[jfeh],eobsfeh)
    eobsmass = np.abs(FMASSERR*modmass)
    obsmass  = np.random.normal(modmass,eobsmass)
    eobsteff = np.abs(FTEFFERR*modteff)
    obsteff  = np.random.normal(modteff,eobsteff)
    eobslogg = np.abs(FLOGGERR*modlogg)
    obslogg  = np.random.normal(modlogg,eobslogg)
    appJ     = am.appMag(modabsJ,s) 
    appH     = am.appMag(modabsH,s)
    appK     = am.appMag(modabsK,s)
    eobsappJ = np.abs(FMAGERR*appJ)
    obsappJ  = np.random.normal(appJ,eobsappJ)
    eobsappH = np.abs(FMAGERR*appH)
    obsappH  = np.random.normal(appH,eobsappH)
    eobsappK = np.abs(FMAGERR*appK)
    obsappK  = np.random.normal(appK,eobsappK)
    
    # Create array with model values and observed values 
    output[jmock,0] = np.copy(isoagegrid[jage])
    output[jmock,1] = np.copy(isomhgrid[jfeh])
    output[jmock,2] = np.copy(modmass)
    output[jmock,3] = np.copy(dist[jmock])
    output[jmock,4] = np.copy(obsl)
    output[jmock,5] = np.copy(eobsl)
    output[jmock,6] = np.copy(obsb)
    output[jmock,7] = np.copy(eobsb)
    output[jmock,8] = np.copy(obsvarpi)
    output[jmock,9] = np.copy(eobsvarpi)
    output[jmock,10] = np.copy(obsfeh)
    output[jmock,11] = np.copy(eobsfeh)
    output[jmock,12] = np.copy(obsmass)
    output[jmock,13] = np.copy(eobsmass)
    output[jmock,14] = np.copy(obsteff)
    output[jmock,15] = np.copy(eobsteff)
    output[jmock,16] = np.copy(obslogg)
    output[jmock,17] = np.copy(eobslogg)
    output[jmock,18] = np.copy(obsappJ)
    output[jmock,19] = np.copy(eobsappJ)
    output[jmock,20] = np.copy(obsappH)
    output[jmock,21] = np.copy(eobsappH)
    output[jmock,22] = np.copy(obsappK)
    output[jmock,23] = np.copy(eobsappK)
    
    """
    # Use exact values
    output[jmock,4] = np.copy(l[jmock])
    output[jmock,5] = np.copy(eobsl)
    output[jmock,6] = np.copy(b[jmock])
    output[jmock,7] = np.copy(eobsb)
    output[jmock,8] = np.copy(modvarpi)
    output[jmock,9] = np.copy(eobsvarpi)
    output[jmock,10] = np.copy(feh[jmock])
    output[jmock,11] = np.copy(eobsfeh)
    output[jmock,12] = np.copy(modmass)
    output[jmock,13] = np.copy(eobsmass)
    output[jmock,14] = np.copy(modteff)
    output[jmock,15] = np.copy(eobsteff)
    output[jmock,16] = np.copy(modlogg)
    output[jmock,17] = np.copy(eobslogg)
    output[jmock,18] = np.copy(appJ)
    output[jmock,19] = np.copy(eobsappJ)
    output[jmock,20] = np.copy(appH)
    output[jmock,21] = np.copy(eobsappH)
    output[jmock,22] = np.copy(appK)
    output[jmock,23] = np.copy(eobsappK)
    """

In [5]:
## Write to file
outputDF = pd.DataFrame(output,columns=["modage","modfeh","modmass","moddist",\
                                        "obsl","eobsl","obsb","eobsb","obsvarpi","eobsvarpi",
                                        "obsfeh","eobsfeh","obsmass","eobsmass","obsteff","eobsteff",
                                        "obslogg","eobslogg","obsappJ","eobsappJ",
                                        "obsappH","eobsappH","obsappK","eobsappK"])
outputDF.to_csv(MOCKDATAFILE)